# Discovering Earth Observation Data with the EO-MQS

The EO-MQS service is hosted within the C-SCALE federated cloud infrastructure and provides a unified way of discovering Copernicus data available within the federation by making use of the SpatioTemporal Asset Catalog (STAC) specification. The purpose of this notebook is to prvovide a concise introduction on how to use open-source Python libraries to search for geospatial data exposed by the EO-MQS STAC API.

## Prerequisites

In this example, we are going to make use of a popular STAC client for Python, the `pystac-client`. The library is already installed in this environment, but can be manually installed anywhere else via `pip install pystac-client`. 
Alternatively, common Python libraries like the `requests` library which support working with HTTP APIs are of course also well suited.

To get started, we need to import the `Client` class to connect to the EO-MQS which exposes its STAC API under `https://mqs.eodc.eu/stac/v1`.

In [1]:
from pystac_client import Client

client = Client.open("https://mqs.eodc.eu/stac/v1")


In [2]:
client.title

'C-SCALE Earth Observation Metadata Query Service (EO-MQS)'

## CollectionClient

The client can be used to iterate through the Collections available in the EO-MQS Catalog. 

The `get_collections` method fetches the collections from the `/collections` endpoint and returns an iterable. To load a particular collection for further use we call the `get_collection(<collection_id>)` method below.

In [3]:
for collection in client.get_collections():
    print(collection)

<CollectionClient id=EODC|sentinel1-grd>
<CollectionClient id=EODC|sentinel-2-l1c>
<CollectionClient id=EODC|s1-global-sigma0>
<CollectionClient id=EODC|s1-demo-sigma0>
<CollectionClient id=EODC|landsat-c2-l1>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-grd>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-ocn>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-raw>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-slc>
<CollectionClient id=GRNET-OPENSTACK|sentinel-2-l1b>
<CollectionClient id=GRNET-OPENSTACK|sentinel-2-l1c>
<CollectionClient id=GRNET-OPENSTACK|sentinel-2-l2a>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-olci-l1b>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-olci-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-slstr-l1b>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-slstr-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-stm-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-syn-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-5p-l1b>
<CollectionClient id=

On static as well as dynamic catalogues we cann also make use of the `links` attributes which lets us quickly examinate, for instance, the number of available collections.

In [4]:
child_links = client.get_links('child')
print(f"The EO-MQS currently features {len(child_links)} collections.")

The EO-MQS currently features 130 collections.


In [5]:
collection = client.get_collection("VITO|urn:eop:VITO:ESA_WorldCover_10m_2020_V1")
#collection = client.get_collection("sentinel-2-l1c")
collection

id: VITO|urn:eop:VITO:ESA_WorldCover_10m_2020_V1
title: ESA WorldCover products 10 meter COG format
description: The WorldCover product will be released per 3 x 3 degree tile.
type: Collection
id: urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E006
"bbox: [6, 0, 9, 3]"
datetime: 2020-12-31T23:59:59Z
title: ESA_WorldCover_10m_2020_v100_N00E006
created: 2021-10-13T06:48:53Z
updated: 2021-10-14T21:57:35Z
start_datetime: 2020-01-01T00:00:00Z


There are many ways to access the collection metadata programmatically. 

In [6]:
print(f"This collection contains data in the following temporal inteval: {collection.extent.temporal.to_dict()}")

This collection contains data in the following temporal inteval: {'interval': [['2021-06-01T00:00:00Z', None]]}


In [7]:
# To verify this extent, we can calculate the actual limits like this:
collection.update_extent_from_items()

In [8]:
collection.extent.temporal.to_dict()

{'interval': [['2020-01-01T00:00:00Z', '2020-12-31T23:59:59Z']]}

In [9]:
# Check which STAC Extensions are used by the collection
collection.stac_extensions

[]

## STAC Items

Simlarly to before, we can use the collection client instance to iterate over the items contained in the collection. The server must provide the `/collections/<collection_id>/items` endpoint to support this feature automatically. This can be useful to manually filter items or extract information programmatically. The `get_all_items()` method again returns an iterator.

In [10]:
items = collection.get_all_items()

In [15]:
# Load 10 items
items10 = []
for n, item in enumerate(items):
    if len(items10) == 10:
        break
    print(f"Append item {item.id}")
    items10.append(item)

Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E015
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E018
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E021
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E024
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E027
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E030
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E033
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E036
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E039
Append item urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E042


In [16]:
items10[-1]

id: urn:eop:VITO:ESA_WorldCover_10m_2020_V1:ESA_WorldCover_10m_2020_v100_N00E042
"bbox: [42, 0, 45, 3]"
datetime: 2020-12-31T23:59:59Z
title: ESA_WorldCover_10m_2020_v100_N00E042
created: 2021-10-13T06:53:05Z
updated: 2021-10-14T21:57:47Z
start_datetime: 2020-01-01T00:00:00Z
end_datetime: 2020-12-31T23:59:59Z
href: https://services.terrascope.be/download/WORLDCOVER/ESA_WORLDCOVER_10M_2020_V100/MAP/ESA_WorldCover_10m_2020_v100_N00E042_Map/ESA_WorldCover_10m_2020_v100_N00E042_InputQuality.tif
type: image/tiff
title: ESA_WORLDCOVER_10M_INPUTQUALITY


In [17]:
# If the item provides a previeww image we can look at it in here using the following code
from IPython.display import Image

Image(url=items10[-1].assets["ESA_WORLDCOVER_10M_MAP"].href, width=500)

KeyError: 'thumbnail-png'

## Item Search

Data providers that have realized their STAC implementation in terms of a dynamic STAC API offer users the opportunity to search their Catalogs using spatial and temporal constraints. The `pystac_client` enables this search via the class method `search`. This function returns an ItemSearch instance that can further be accessed to retrieve matched items.

Note that in its current implementation, the EO-MQS supports the *core* STAC search endpoint paramters as described in the [STAC API - Item Search](https://github.com/radiantearth/stac-api-spec/tree/master/item-search#query-parameter-table) specification. Those are:
- limit
- bbox
- datetime
- intersects
- ids
- collections


### Example 1: Search for Sentinel-1 GRD data over Austria (bbox) in 2022

This first example makes use of the `bbox`, `datetime` and the `collections` parameters. Learn about the correct formatting of these values on the STAC Spec GitHub page or by looking at the [pystac-client docs](https://pystac-client.readthedocs.io/en/latest/api.html#item-search).

In [ ]:
# We can iterate and grep the collections automatically or look it up in the browser or API
s1_collections = []
for collection in client.get_collections():
    if "grd" in collection.id.lower() :
        print(f"Append collection {collection.id} to list of Sentinel-1 collections.")
        s1_collections.append(collection.id)

# manually add collections as requried
s1_collections.append("CREODIAS|SENTINEL-1")

If you do not have bbox coordinates at hand, you can quickly create your region of interest at [geojson.io](https://geojson.io).

In [ ]:
bbox_aut = [9.25, 46.31, 17.46, 49.18]
time_period = "2022-01-01/2022-10-31"
limit = 20 # limit the number of items to be returned (per data provider)

In [ ]:
# put together the search dictionary
search1 = {"collections": s1_collections,
           "bbox": bbox_aut,
           "datetime": time_period,
           "limit": limit}    

In [ ]:
results1 = client.search(**search1)

In [ ]:
items = results1.item_collection()

print(f"We found {len(items)} matching items.")

#### Tips on how to increase match rate

**NOTE:** Depending on the backend implementation, the `collections` parameter might have restrictions on allowed values. To make sure all possible items are fetched, consider iterating over the collections in our list and issue separate requests.

In [ ]:
items_list = []
for s1_collection in s1_collections:
    results = client.search(collections=[s1_collection], 
                            bbox=bbox_aut, 
                            datetime=time_period, 
                            limit=limit)
    try:
        items_list.extend(results.item_collection())
    except:
        print(f"Search for items with collection id {s1_collection} failed or no items found.")


In [ ]:
print(f"Now, we found {len(items_list)} matching items.")

**NOTE:** By default, `pystac_client.search` will choose the HTTP method *POST* when making requests to the STAC API. Some data providers will only allow *GET* requests! Potentially available datasets might therefore remain undetected. *Hint: specify the method explicitly.*

In [ ]:
items_list = []
for s1_collection in s1_collections:
    results = client.search(collections=[s1_collection], 
                            bbox=bbox_aut, 
                            datetime=time_period, 
                            limit=limit,
                            method="GET")
    try:
        items_list.extend(results.item_collection())
    except:
        print(f"Search for items with collection id {s1_collection} failed.")


In [ ]:
print(f"Now, we found {len(items_list)} matching items.")

In [ ]:
print(items_list[0].id)
Image(url=items_list[0].assets["thumbnail"].href, width=500)

### Example 2: Search for Sentinel-2 data intersecting a GeoJSON object

The second example makes use of the `intersects` and the `collections` parameters. Note that you cannot specify both `bbox` and `intersects`, this will result in an error.

We make use of a geojson file located in the data folder.

In [ ]:
import json

with open('../data/portugal_spain.geojson') as f:
    geom = json.load(f)

In [ ]:
geom

In [ ]:
s2_collections = []
for collection in client.get_collections():
    if "l1c" in collection.id.lower() :
        print(f"Append collection {collection.id} to list of Sentinel-2 L1C collections.")
        s2_collections.append(collection.id)


We pick one that supports POST requests, for instance *VITO|urn:eop:VITO:CGS_S2_L1C*.

In [ ]:
search2 = {"collections": ["VITO|urn:eop:VITO:CGS_S2_L1C"],
           "intersects": geom,
           "limit": limit,
           "method": "POST"}    

In [ ]:
results2 = client.search(**search2)

In [ ]:
items = results2.item_collection()

print(f"We found {len(items)} matching items.")

In [ ]:
items[0]

NOTE: You can always visualize STAC data (collections, items, etc.) in external tools like the STAC Browser, for instance do the following:

In [ ]:
print(f"Look at this item in the STAC Browser: https://radiantearth.github.io/stac-browser/#/external/{items[0].get_self_href()}")

## Example 3: Use non-default search parameters (experimental!)

As mentioned, the EO-MQS officially does not support parameters that are not part of the STAC API core specifications. However, when realizing a STAC implementation at a data provider's site, we usually make use of open-source libraries that are constantly being developed and improved. An example for such an improvement is the addition of search paramters like `filter`, `sortby` or `fields`.

This section hints at what can be done using these additional parameters.

In [ ]:
# let's re-use a sentinel-2 collection from before
search3 = {"collections": s2_collections[0],
           "limit": limit,
           "method": "GET"}    

In [ ]:
# sort the results in a descending manner based on the datetime property
sortby = "-properties.datetime"
search_sort = search3
search_sort["sortby"] = sortby

In [ ]:
results3 = client.search(**search_sort)
items = results3.item_collection()

In [ ]:
print(f"First item {items[0]} has datetime {items[0].get_datetime()}")
print(f"Lastt item {items[-1]} has datetime {items[-1].get_datetime()}")

In [ ]:
# filter based on specific property, e.g. cloud cover
filterby = {
  "filter": {
    "op": "and",
    "args": [
      {
        "op": "<",
        "args": [
          {
            "property": "eo:cloud_cover"
          },
          10
        ]
      }
    ]
  }
}
search_filter = search3
search_filter["sortby"] = "-properties.eo:cloud_cover"
search_filter["filter"] = filterby
search_filter["method"] = "POST"
search_filter["limit"] = 100

In [ ]:
results3 = client.search(**search_filter)
items = results3.item_collection()

In [ ]:
print(f"First item {items[0]} has cloud cover {items[0].properties.get('eo:cloud_cover')}%")
print(f"Last item {items[-1]} has cloud cover {items[-1].properties.get('eo:cloud_cover')}%")
